In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch.utils.data import DataLoader

from pose_estimation import PoseData, PoseDataNPZ, PoseDataNPZTorch, COLOR_PALETTE

WORKDIR = f"{os.getcwd()}/.."
DATA_FOLDER = os.path.join(WORKDIR, "data_folder")
DATASET_NPZ_PATH = os.path.join(DATA_FOLDER, "dataset_npz")


In [2]:
dataset = None
dataset = PoseDataNPZTorch(DATASET_NPZ_PATH)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)


Presumed Preloaded NPZ Dataset: /Users/armanommid/Code/CSE/CSE275/HW2/XSwinDiffusion/../data_folder/dataset_npz


In [3]:
points = []

for i, (s, t, p) in enumerate(dataloader):
    
    print(s.dtype, t.dtype)

    if i % 10 == 0:
        print(i)


True
torch.float32 torch.float32
0
True
torch.float32 torch.float32
True
torch.float32 torch.float32
True
torch.float32 torch.float32
True
torch.float32 torch.float32
True
torch.float32 torch.float32
True
torch.float32 torch.float32
True
torch.float32 torch.float32
True
torch.float32 torch.float32
True
torch.float32 torch.float32
True
torch.float32 torch.float32
10
True
torch.float32 torch.float32


KeyboardInterrupt: 